#  Matching Users

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from Utils import TransantiagoConstants

first_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/1st_quarter.xlsx"
second_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/2nd_quarter.xlsx"
third_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/3rd_quarter.xlsx"
codes_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/04_DTPM/codes_services.xlsx"

first_quarter_evasion = pd.read_excel(first_quarter_path, encoding = 'latin-1')
second_quarter_evasion = pd.read_excel(second_quarter_path, encoding = 'latin-1')
third_quarter_evasion = pd.read_excel(third_quarter_path, encoding = 'latin-1')
codes = pd.read_excel(codes_path, encoding = 'latin-1')

first_quarter_evasion['TIEMPO'] = first_quarter_evasion['HORA'].astype(str)+':'+first_quarter_evasion['MINUTOS'].astype(str)+':00'
second_quarter_evasion['TIEMPO'] = second_quarter_evasion['HORA'].astype(str)+':'+second_quarter_evasion['MINUTOS'].astype(str)+':00'
third_quarter_evasion['TIEMPO'] = third_quarter_evasion['HORA'].astype(str)+':'+third_quarter_evasion['MINUTOS'].astype(str)+':00'

first_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
second_quarter_evasion.columns = ['FECHA', 'SERVICIO','TIPO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
third_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
del second_quarter_evasion['TIPO']

#Processing codes before merging.

codes_ida = codes[codes['DIRECTION']=='Ida']
codes_ret = codes[codes['DIRECTION']=='Ret']

codes_ida = codes_ida.rename(columns = {'USER_CODE':'SERVICIO'})
codes_ret = codes_ret.rename(columns = {'USER_CODE':'SERVICIO'})

del codes_ida['DIRECTION']
del codes_ret['DIRECTION']

In [2]:
print(type(first_quarter_evasion.loc[0,'TIEMPO'])) #should be a string
print(type(first_quarter_evasion.loc[0,'FECHA'])) #should be a timestamp

<class 'str'>
<class 'pandas._libs.tslib.Timestamp'>


In [3]:
frames = [first_quarter_evasion, second_quarter_evasion, third_quarter_evasion]
evasion = pd.concat(frames, keys=['first', 'second', 'third'])
del evasion['LUGAR_INICIO']

* Merge should be made only by codes_ida or codes_ret. Be aware of it

In [4]:
evasion = pd.merge(evasion,codes_ida, on=['SERVICIO'], how='left')

In [5]:
print(evasion.loc[evasion['TS_CODE'].isnull(),'SERVICIO'].unique()) #should be only D06, so the line below makes sense. Anyway, it is hardcoded so be aware.
evasion.loc[evasion['TS_CODE'].isnull(),'TS_CODE']='446'

['D06']


* Processing evasion-ddbb before merging

In [6]:
evasion['PATENTE'] =  evasion['PATENTE'].str.replace(' ','')

evasion['SERVICIO_TMP'] = evasion['SERVICIO'].apply(str)
evasion['TS_CODE_TMP'] = evasion['TS_CODE'].apply(str)

del evasion['SERVICIO']
del evasion['TS_CODE']

evasion = evasion.rename(columns = {'SERVICIO_TMP':'SERVICIO_USUARIO', 'TS_CODE_TMP':'SERVICIO'})
evasion_paradero = evasion[evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]
print(type(evasion['SERVICIO'][0])) #should be a string

<class 'str'>


In [7]:
print('Number of rows in complete evasion database is: ' + str(len(evasion.index)))
print('Number of rows in evasion in paradero database is: ' + str(len(evasion_paradero.index)))
print('Number of rows in evasion in paradero by first door database is: ' + str(len(evasion_paradero_first.index)))

Number of rows in complete evasion database is: 99422
Number of rows in evasion in paradero database is: 97706
Number of rows in evasion in paradero by first door database is: 34881


* Getting rid of duplicated rows

In [8]:
#Getting duplicated rows
duplicated_evasion_paradero_first = evasion_paradero_first[evasion_paradero_first.duplicated(['FECHA','PATENTE','PUERTAS','N_PUERTA','HORA_INICIO','HORA','MINUTO','TP','TIEMPO','UN','SERVICIO_USUARIO','SERVICIO'], keep=False)]
print('Number of duplicated rows in evasion in paradero by first door database is: ' + str(len(duplicated_evasion_paradero_first.index)))

#Colapsing duplicated rows
grouped_temporal_evasion_paradero_first = duplicated_evasion_paradero_first.groupby(['FECHA','PATENTE','PUERTAS','N_PUERTA','HORA_INICIO','HORA','MINUTO','TP','TIEMPO','UN','SERVICIO_USUARIO','SERVICIO'])['INGRESAN','NO_VALIDAN'].sum()
grouped_temporal_evasion_paradero_first = grouped_temporal_evasion_paradero_first.reset_index()
print('Number of collapsed-duplicated rows in evasion in paradero by first door database is: ' + str(len(grouped_temporal_evasion_paradero_first.index)))

#Deleting duplicated rows in evasion in paradero by first door database
non_duplicated_evasion_paradero_first = evasion_paradero_first.drop_duplicates(['FECHA','PATENTE','PUERTAS','N_PUERTA','HORA_INICIO','HORA','MINUTO','TP','TIEMPO','UN','SERVICIO_USUARIO','SERVICIO'], keep=False)
print('Number of rows in evasion in paradero by first door without duplicated rows at all is: ' + str(len(non_duplicated_evasion_paradero_first.index)))

#Appendind
del evasion_paradero_first
frames = [non_duplicated_evasion_paradero_first, grouped_temporal_evasion_paradero_first]
evasion_paradero_first = pd.concat(frames)
print('Final number of rows in evasion in paradero by first door with collapsed duplicated rows is: ' + str(len(evasion_paradero_first.index)))

Number of duplicated rows in evasion in paradero by first door database is: 2246
Number of collapsed-duplicated rows in evasion in paradero by first door database is: 1120
Number of rows in evasion in paradero by first door without duplicated rows at all is: 32635
Final number of rows in evasion in paradero by first door with collapsed duplicated rows is: 33755


* Filtering dates not in common dates

In [9]:
common_dates = TransantiagoConstants.common_dates
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]
common_dates_evasion = evasion_paradero_first[evasion_paradero_first['FECHA'].isin(common_dates_timestamp)]
len(common_dates_evasion.index) #Remember to store the number.

14756

* <font color='red'>Filtering for one specific common date and analyze it to refactor and debug <- this part should be omitted in the final script. Above methodology is general</font>

In [10]:
example = common_dates_evasion[common_dates_evasion['FECHA']==pd.to_datetime('2017-04-11')]

In [11]:
len(example.index)

1106

In [12]:
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [13]:
date = '2017-04-11'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
sorted_df = etapas_builder.runRawProcess()

Not found in turnstile database: 1829611


In [14]:
len(sorted_df.index)

2598131

* Processing sorted_df before reducing complexity

In [15]:
sorted_df['servicio_subida'] =  sorted_df['servicio_subida'].str.replace('T','')
sorted_df['servicio_subida'] =  sorted_df['servicio_subida'].str.split(' ').str[0]

In [16]:
sorted_df['sitio_subida'] = sorted_df['sitio_subida'].str.replace("-", "")
sorted_df['sitio_subida'] = sorted_df['sitio_subida'].str.replace(" ", "")

* First: filter etapas by patentes and servicios surveyed in evasion ddbb so to reduce search space

In [17]:
patentes = example['PATENTE'].unique()
patentes = patentes.tolist()
servicios = example['SERVICIO'].unique()
servicios = servicios.tolist()

In [18]:
clean_sorted_df = sorted_df[(sorted_df['sitio_subida'].isin(patentes))&(sorted_df['servicio_subida'].isin(servicios))]

In [20]:
sorted_df.head()

,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion
218463,88993254,1,BUS,2017-04-11 03:20:01,F28,L-34-52-65-OP,BBJZ70,NaT
967981,1520963284,1,BUS,2017-04-11 05:32:26,F01,L-34-89-2-PO,BBJZ70,NaT
962527,1507480586,1,BUS,2017-04-11 05:32:29,F01,L-34-89-2-PO,BBJZ70,NaT
2403055,3875759531,1,BUS,2017-04-11 05:32:31,F01,L-34-89-2-PO,BBJZ70,NaT
1969205,3214208356,2,BUS,2017-04-11 05:32:44,F01,L-34-89-2-PO,BBJZ70,NaT


In [21]:
len(clean_sorted_df.index)

11937

* Evasion should be sorted first

In [22]:
example['TIEMPO'] = example['FECHA'].dt.strftime('%Y-%m-%d') + ' ' + example['TIEMPO']

C:\Users\leoca_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
example['TIEMPO'] = pd.to_datetime(example['TIEMPO'])

C:\Users\leoca_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
example = example.sort_values(by=['PATENTE', 'SERVICIO', 'TIEMPO'], ascending=[True, True, True])

In [25]:
example = example.reset_index(drop =True)

In [26]:
example.head(8)

,FECHA,HORA,HORA_INICIO,INGRESAN,MINUTO,NO_VALIDAN,N_PUERTA,PATENTE,PUERTAS,SERVICIO,SERVICIO_USUARIO,TIEMPO,TP,UN
0,2017-04-11,15,15:10:00,5,10,0,1,BJFB62,3,507,507,2017-04-11 15:10:00,P,5.0
1,2017-04-11,15,15:10:00,0,12,0,1,BJFB62,3,507,507,2017-04-11 15:12:00,P,5.0
2,2017-04-11,15,15:10:00,2,16,1,1,BJFB62,3,507,507,2017-04-11 15:16:00,P,5.0
3,2017-04-11,15,15:10:00,2,18,1,1,BJFB62,3,507,507,2017-04-11 15:18:00,P,5.0
4,2017-04-11,15,15:10:00,5,19,4,1,BJFB62,3,507,507,2017-04-11 15:19:00,P,5.0
5,2017-04-11,15,15:10:00,2,21,2,1,BJFB62,3,507,507,2017-04-11 15:21:00,P,5.0
6,2017-04-11,15,15:10:00,8,23,0,1,BJFB62,3,507,507,2017-04-11 15:23:00,P,5.0
7,2017-04-11,15,15:10:00,3,27,1,1,BJFB62,3,507,507,2017-04-11 15:27:00,P,5.0


In [27]:
clean_sorted_df[(clean_sorted_df['sitio_subida']=='BJFB62')&(clean_sorted_df['servicio_subida']=='507')&(clean_sorted_df['t_subida']>=pd.to_datetime('2017-04-11 15:08:00'))].head(15)

,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion
2455144,3921423867,5,BUS,2017-04-11 15:09:21,507,L-32-11-45-NS,BJFB62,NaT
1981835,3249105194,5,BUS,2017-04-11 15:09:26,507,L-32-11-45-NS,BJFB62,NaT
2145745,3540430542,5,BUS,2017-04-11 15:09:27,507,L-32-11-45-NS,BJFB62,NaT
2547799,4011394493,5,BUS,2017-04-11 15:09:29,507,L-32-11-45-NS,BJFB62,NaT
1212807,2166667651,1,BUS,2017-04-11 15:16:15,507,T-32-156-OP-20,BJFB62,NaT
885328,1292305198,1,BUS,2017-04-11 15:19:20,507,T-32-156-OP-45,BJFB62,NaT
2166031,3543518011,3,BUS,2017-04-11 15:20:21,507,T-32-156-OP-50,BJFB62,NaT
625604,670787563,4,BUS,2017-04-11 15:23:27,507,E-18-156-OP-5,BJFB62,NaT
1167151,2118814731,2,BUS,2017-04-11 15:23:30,507,E-18-156-OP-5,BJFB62,NaT
185839,52625842,1,BUS,2017-04-11 15:23:32,507,E-18-156-OP-5,BJFB62,NaT


In [28]:
clean_sorted_df = clean_sorted_df.reset_index(drop=True)

In [29]:
clean_sorted_df.head()

,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion
0,3854261808,1,BUS,2017-04-11 08:00:54,507,L-10-55-5-PO,BJFB62,NaT
1,3857420299,1,BUS,2017-04-11 08:02:49,507,L-10-82-5-PO,BJFB62,NaT
2,1182024974,1,BUS,2017-04-11 08:03:46,507,L-10-37-20-NS,BJFB62,NaT
3,3562465182,2,BUS,2017-04-11 08:03:48,507,L-10-37-20-NS,BJFB62,NaT
4,679843243,1,BUS,2017-04-11 08:06:16,507,L-10-21-45-NS,BJFB62,NaT


* New Methodology: this methodology is based on appending PARADEROS to evasion ddbb

In [32]:
example.head()

,FECHA,HORA,HORA_INICIO,INGRESAN,MINUTO,NO_VALIDAN,N_PUERTA,PATENTE,PUERTAS,SERVICIO,SERVICIO_USUARIO,TIEMPO,TP,UN,start_cut,end_cut
0,2017-04-11,15,15:10:00,5,10,0,1,BJFB62,3,507,507,2017-04-11 15:10:00,P,5.0,,
1,2017-04-11,15,15:10:00,0,12,0,1,BJFB62,3,507,507,2017-04-11 15:12:00,P,5.0,,
2,2017-04-11,15,15:10:00,2,16,1,1,BJFB62,3,507,507,2017-04-11 15:16:00,P,5.0,,
3,2017-04-11,15,15:10:00,2,18,1,1,BJFB62,3,507,507,2017-04-11 15:18:00,P,5.0,,
4,2017-04-11,15,15:10:00,5,19,4,1,BJFB62,3,507,507,2017-04-11 15:19:00,P,5.0,,


In [ ]:
i = 0
past_plate = '-'
past_service = '-'

example['start_cut']=''
example['end_cut']=''
example['paradero']=''
past_util_df = pd.DataFrame()
past_par_subida = ['']
from IPython.core.debugger import Tracer

for index,row in example.iterrows():
    future_index = index+1
    actual_plate = row['PATENTE']
    actual_service = row['SERVICIO']
    actual_hour = row['TIEMPO']
    
    if (actual_plate!=past_plate)|(actual_service!=past_service):
        past_hour = pd.to_datetime('2017-04-11 00:00:00') #Starting hour <- consider to change this
        
    if past_hour == pd.to_datetime('2017-04-11 00:00:00'): #Assuming a pre-defined value, i.e., 1 minute.
        start_cut = actual_hour - (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        start_cut = actual_hour - (actual_hour - past_hour)/2
    
    if future_index >= len(example.index): #We are at the end of the ddff. Assuming a pre-defined value, i.e., 1 minute.
        end_cut = actual_hour + (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        future_hour = example.loc[future_index,'TIEMPO']
        end_cut = actual_hour + (future_hour - actual_hour)/2
        
    actual_util_df = clean_sorted_df[(clean_sorted_df['sitio_subida']==actual_plate)&(clean_sorted_df['servicio_subida']==actual_service)&((start_cut<=clean_sorted_df['t_subida'])&(clean_sorted_df['t_subida']<=end_cut))]
    actual_par_subida = actual_util_df['par_subida'].unique().tolist()
    
    if len(actual_par_subida)==1:
        example.loc[index,'paradero']=actual_par_subida[0]
    elif (len(actual_par_subida)>1)&(len(past_par_subida)>=1):
        if (past_par_subida[(len(past_par_subida)-1)]==actual_par_subida[0]):
            example.loc[index,'paradero']=actual_par_subida[1]
    
    past_par_subida = actual_par_subida
    
    example.loc[index,'start_cut']=start_cut
    example.loc[index,'end_cut']=end_cut
    past_plate = actual_plate
    past_service = actual_service
    past_hour = actual_hour
    if index>=10:
        Tracer()()
#    if 
    

#    if ((example.loc[index,'PATENTE']=='BJFB62')&(example.loc[index,'SERVICIO']=='507')&(pd.to_datetime('2017-04-11 15:29:00')<=example.loc[index,'TIEMPO'])&(example.loc[index,'TIEMPO']<=pd.to_datetime('2017-04-11 15:35:00'))):
#        print(past_par_subida)
#        print(actual_par_subida)
#        print('separator')
    
        
#    past_util_df = actual_util_df

      
#    util_df_indexes = util_df.index.values
#    number_of_indexes = len(util_df_indexes)
    
#    first_util_df_index = util_df_indexes[0]
#    last_util_df_index = util_df_indexes[number_of_indexes-1]
    #Checking up of util_df --- until a condition is reached
#    j=1
#    while True:
#        upper_util_row = clean_filtered_df.loc(first_util_df_index-j)
#        if upper_util_row['par_subida'] == util_df['par_subida']: #Si el paradero de subida de la fila inmediatamente anterior al util_df es el mismo, then append it and continue < the issue here es que hay que sacarlo del util_df anteriormente analizado...
#            util_df.concat[upper_util_row]
#            j=j+1
#        else 
    
    
#    total_delta = util_df['diferencia_tiempo_secs'].sum()
#    mean_delta = util_df['diferencia_tiempo_secs'].mean()
#    count_delta = util_df['diferencia_tiempo_secs'].count() 
    
#    example.loc[index,'total_delta'] = total_delta
#    example.loc[index,'mean_delta'] = mean_delta
#    example.loc[index,'count_delta'] = count_delta

#    i=i+1
#    if (actual_plate == 'ZN6539')&(actual_service=='109'):
#        print(i)
#        print(start_cut)
#        print(end_cut)

#        break

C:\Users\leoca_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`


> <ipython-input-105-5d3bca505e89>(12)<module>()
     10 from IPython.core.debugger import Tracer
     11 
---> 12 for index,row in example.iterrows():
     13     future_index = index+1
     14     actual_plate = row['PATENTE']

ipdb> example.loc[index]
FECHA               2017-04-11 00:00:00
HORA                                 15
HORA_INICIO                    15:10:00
INGRESAN                              4
MINUTO                               34
NO_VALIDAN                            2
N_PUERTA                              1
PATENTE                          BJFB62
PUERTAS                               3
SERVICIO                            507
SERVICIO_USUARIO                    507
TIEMPO              2017-04-11 15:34:00
TP                                    P
UN                                    5
start_cut           2017-04-11 15:33:00
end_cut             2017-04-11 15:34:30
paradero                 T-18-156-OP-53
Name: 10, dtype: object
ipdb> c


C:\Users\leoca_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  if sys.path[0] == '':


> <ipython-input-105-5d3bca505e89>(12)<module>()
     10 from IPython.core.debugger import Tracer
     11 
---> 12 for index,row in example.iterrows():
     13     future_index = index+1
     14     actual_plate = row['PATENTE']

ipdb> example.loc[index]
FECHA               2017-04-11 00:00:00
HORA                                 15
HORA_INICIO                    15:10:00
INGRESAN                              1
MINUTO                               35
NO_VALIDAN                            1
N_PUERTA                              1
PATENTE                          BJFB62
PUERTAS                               3
SERVICIO                            507
SERVICIO_USUARIO                    507
TIEMPO              2017-04-11 15:35:00
TP                                    P
UN                                    5
start_cut           2017-04-11 15:34:30
end_cut             2017-04-11 15:36:00
paradero                               
Name: 11, dtype: object
ipdb> c
> <ipython-input-105-5d3bca505e89

In [99]:
example.loc[13:16]

,FECHA,HORA,HORA_INICIO,INGRESAN,MINUTO,NO_VALIDAN,N_PUERTA,PATENTE,PUERTAS,SERVICIO,SERVICIO_USUARIO,TIEMPO,TP,UN,start_cut,end_cut,paradero
13,2017-04-11,15,15:10:00,1,39,0,1,BJFB62,3,507,507,2017-04-11 15:39:00,P,5.0,2017-04-11 15:38:00,2017-04-11 15:40:30,
14,2017-04-11,15,15:10:00,10,42,3,1,BJFB62,3,507,507,2017-04-11 15:42:00,P,5.0,2017-04-11 15:40:30,2017-04-11 15:43:00,E-18-156-OP-70
15,2017-04-11,15,15:10:00,2,44,1,1,BJFB62,3,507,507,2017-04-11 15:44:00,P,5.0,2017-04-11 15:43:00,2017-04-11 15:45:00,E-18-156-OP-70
16,2017-04-11,15,15:10:00,7,46,5,1,BJFB62,3,507,507,2017-04-11 15:46:00,P,5.0,2017-04-11 15:45:00,2017-04-11 15:47:00,


In [102]:
clean_sorted_df.loc[72:85]

,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion
72,2164429267,2,BUS,2017-04-11 15:39:55,507,T-18-156-OP-65,BJFB62,NaT
73,81978614,2,BUS,2017-04-11 15:42:39,507,E-18-156-OP-70,BJFB62,NaT
74,1794035998,1,BUS,2017-04-11 15:42:42,507,E-18-156-OP-70,BJFB62,NaT
75,3211081812,1,BUS,2017-04-11 15:42:46,507,E-18-156-OP-70,BJFB62,NaT
76,3575968910,1,BUS,2017-04-11 15:42:52,507,E-18-156-OP-70,BJFB62,NaT
77,3060750619,1,BUS,2017-04-11 15:42:55,507,E-18-156-OP-70,BJFB62,NaT
78,3538608846,2,BUS,2017-04-11 15:42:59,507,E-18-156-OP-70,BJFB62,NaT
79,2510130878,4,BUS,2017-04-11 15:43:02,507,E-18-156-OP-70,BJFB62,NaT
80,2509851950,3,BUS,2017-04-11 15:43:08,507,E-18-156-OP-70,BJFB62,NaT
81,2394856299,2,BUS,2017-04-11 15:45:27,507,T-20-189-OP-5,BJFB62,NaT


In [31]:
par_subida

['T-20-176-SN-40']

* Old Methodology

In [ ]:
i = 0
past_plate = '-'
past_service = '-'

example['total_delta']=''
example['mean_delta']=''
example['count_delta']=''
example['start_cut']=''
example['end_cut']=''

for index,row in example.iterrows():
    future_index = index+1
    actual_plate = row['PATENTE']
    actual_service = row['SERVICIO']
    actual_hour = row['TIEMPO']
    
    if (actual_plate!=past_plate)|(actual_service!=past_service):
        past_hour = pd.to_datetime('2017-04-11 00:00:00') #Starting hour <- consider to change this
        
    if past_hour == pd.to_datetime('2017-04-11 00:00:00'): #Assuming a pre-defined value, i.e., 1 minute.
        start_cut = actual_hour - (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        start_cut = actual_hour - (actual_hour - past_hour)/2
    
    if future_index >= len(example.index): #We are at the end of the ddff. Assuming a pre-defined value, i.e., 1 minute.
        end_cut = actual_hour + (pd.to_datetime('2017-04-11 00:02:00') - pd.to_datetime('2017-04-11 00:01:00'))
    else:
        future_hour = example.loc[future_index,'TIEMPO']
        end_cut = actual_hour + (future_hour - actual_hour)/2  

    util_df = clean_filtered_df[(clean_filtered_df['sitio_subida']==actual_plate)&(clean_filtered_df['servicio_subida']==actual_service)&((start_cut<=clean_filtered_df['t_subida'])&(clean_filtered_df['t_subida']<=end_cut))]
    util_df_indexes = util_df.index.values
    number_of_indexes = len(util_df_indexes)
    
    first_util_df_index = util_df_indexes[0]
    last_util_df_index = util_df_indexes[number_of_indexes-1]
    #Checking up of util_df --- until a condition is reached
    j=1
    while True:
        upper_util_row = clean_filtered_df.loc(first_util_df_index-j)
        if upper_util_row['par_subida'] == util_df['par_subida']: #Si el paradero de subida de la fila inmediatamente anterior al util_df es el mismo, then append it and continue < the issue here es que hay que sacarlo del util_df anteriormente analizado...
            util_df.concat[upper_util_row]
            j=j+1
        else 
    
    
    total_delta = util_df['diferencia_tiempo_secs'].sum()
    mean_delta = util_df['diferencia_tiempo_secs'].mean()
    count_delta = util_df['diferencia_tiempo_secs'].count() 
    
    example.loc[index,'total_delta'] = total_delta
    example.loc[index,'mean_delta'] = mean_delta
    example.loc[index,'count_delta'] = count_delta
    example.loc[index,'start_cut']=start_cut
    example.loc[index,'end_cut']=end_cut
    
    past_plate = actual_plate
    past_service = actual_service
    past_hour = actual_hour
    
    i=i+1
#    if (actual_plate == 'ZN6539')&(actual_service=='109'):
#        print(i)
#        print(start_cut)
#        print(end_cut)

#        break

# TEST OLD METHODOLOGY (Check issue #8)

In [ ]:
example.head(10)

* Above table shows:
- PATENTE == BJFB62 & TIEMPO == 2017-04-11 15:10:00 & SERVICIO == 507 > sum = 08, mean = 2.66667, count = 3 > check them in clean_filtered_df.loc[182369:182380] > <font color='green'>Test passed</font>
- PATENTE == BJFB62 & TIEMPO == 2017-04-11 15:32:00 & SERVICIO == 507 > sum = 13, mean = 6.50000, count = 2 > check them in clean_filtered_df.loc[182388:182395] > <font color='green'>Test passed, although incorrectly assignment</font>

In [ ]:
clean_filtered_df.loc[182369:182380]

In [ ]:
clean_filtered_df.loc[182388:182395]

In [ ]:
example.tail(10)

* Above table shows:
- PATENTE == ZN6539 & TIEMPO == 2017-04-11 14:31:00 & SERVICIO == 109 > sum = 08, mean = 04, count = 02 > check them in clean_filtered_df.loc[2527175:2527180] > <font color='green'>Test passed</font>
- PATENTE == ZN6539	& TIEMPO == 2017-04-11 14:37:00 & SERVICIO == 109 > sum = 21, mean = 21, count = 01 > check them in clean_filtered_df.loc[2527178:2527187] > <font color='green'>Test passed</font>

In [ ]:
clean_filtered_df.loc[2527175:2527180]

In [ ]:
clean_filtered_df.loc[2527178:2527187]

In [ ]:
util_df

In [ ]:
df_indexes = util_df.index.values

In [ ]:
length = len(df_indexes)

In [ ]:
df_indexes[length-1]